In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [ ]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

In [ ]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [ ]:
data.head()

In [ ]:
##encode categorical variable
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
##One hot encode Geography
from sklearn.preprocessing import OneHotEncoder
one_hot_geo=OneHotEncoder()
geo_encoder=one_hot_geo.fit_transform(data[['Geography']])
geo_encoder

In [ ]:
geo_encoder.toarray()

In [ ]:
one_hot_geo.get_feature_names_out(['Geography'])

In [ ]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_geo.get_feature_names_out(['Geography']))
geo_encoder_df

In [ ]:
#combine
data=pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)

data.head()


In [ ]:
#save the ncoders and scalar
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('one_hot_geo.pkl','wb') as file:
    pickle.dump(one_hot_geo,file)



In [ ]:
data.head()

In [ ]:
x=data.drop('Exited',axis=1)
y=data['Exited']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

scalar=StandardScaler()
x_train=scalar.fit_transform(x_train)
x_test=scalar.transform(x_test)

In [ ]:
with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense




In [ ]:
#Build a ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ##HL 1 connected to input layer
    Dense(32,activation='relu'),##HL2
    Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [ ]:
##compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
##setup the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="log/fit/"+datetime.datetime.now().strftime("%Y%m%d-%h%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
#setup early stopping
EarlyStopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,EarlyStopping_callback]
)

In [ ]:
model.save('model.h5')

In [ ]:
#load tensor board extension
%load_ext tensorboard

In [ ]:
###%tensorboard --logdir log/fit/

In [ ]:
###load the pickle file
